In [12]:
%matplotlib inline

# подключение библиотек
import matplotlib.pyplot as plt
import numpy as np
import copy

# формат вывода чисел
np.set_printoptions(precision=3, suppress=True, formatter={'all': lambda x: f'{x:0.6f}'}) 

In [3]:
# поиск главного элемента в матрице
def find_main_element(A, iteration):
    size = A.shape[0]
    main_element = A[iteration,iteration]
    i_main = iteration
    j_main = iteration

    for i in range(size - iteration): 
        for j in range(size - iteration):
            if abs(A[i + iteration,j + iteration]) > abs(main_element):
                i_main = i + iteration
                j_main = j + iteration
                main_element = A[i_main,j_main]
    return [i_main, j_main]

In [4]:
# Функция для обмена строк
def swap_rows(A, i, j):
    A[[i, j], :] = A[[j, i], :]

# Функция для обмена столбцов
def swap_columns(A, i, j):
    A[:, [i, j]] = A[:, [j, i]]
    
#Функция для обмена значений
def swap(a,b):
    a = a + b
    b = a - b
    a = a - b


In [5]:
def gauss(a, f):
    
    A = copy.deepcopy(a)
    F = copy.deepcopy(f)
    
    size = len(A)  # размеры матрицы - количество переменных Х
    
    X = np.arange(size) # массив для смены порядка переменных Х
        
    for iter in range(size): # прямой ход
        
        main_indices = find_main_element(A,iter) # поиск координат главного элемента
        main_element = A[main_indices[0], main_indices[1]] # галвный элемент
        
        # если главный элемент не стоит в верхней строке, делаем перестановку строк
        if(main_indices[0] - iter != 0): 
            swap_rows(A[iter:, iter:], 0, main_indices[0] - iter)
            swap(F[iter], F[main_indices[0]])
            
        # если главный элемент не стоит в левом столбце,  делаем перестановку столбцов
        if(main_indices[1] - iter != 0): 
            swap_columns(A, iter, main_indices[1])
            swap(X[iter], X[main_indices[1]])
        
        # делим строчку на главный элемент
        A[iter:, iter:][0] =  A[iter:, iter:][0] / main_element #
        F[iter] = F[iter] / main_element
        
        # вычитаем из всех строк первую домноженную на соответсвующий множитель
        for i in range(size - iter - 1):
            F[iter + i + 1] -= (F[iter]  * A[iter:, iter:][i + 1][0])
            A[iter:, iter:][i + 1] -= (A[iter:, iter:][0] * A[iter:, iter:][i + 1][0])

    # приводим правый нижний элемент к 1
    F[-1] /= A[-1, -1]
    A[size - 1, size - 1] = 1
 
    U  = np.ones((size, 1)) # массив для решений СЛАУ
    
    # обратный ход
    for i in range(size-1, -1, -1):
        U[i] = F[i]
        for j in range(i + 1, size):
            U[i] -= U[j] * A[i][j]

    # перестановка переменных в изначальном порядке
    result = np.ones((size, 1))
    for i in range(size):
        result[int(X[i])][0] = U[i][0]

    # возвращаем результат
    return result



In [ ]:
a = 10
size = 5

A = np.eye(size) * (a - 1) + np.ones(size)

F = np.zeros((size, 1))
for i in range(size): F[i] = i + 1


print("A is \n", A)
print("F is \n", F)

U = gauss(A, F)

print("U is \n", U)

Res = A.dot(U)
print("A * U = \n", Res)

In [ ]:
X = np.random.rand(500,500)

Z = np.zeros((500, 1))
for i in range(500): Z[i] = i + 1


print("X is ", X)
print("Z is ", Z)

R = gauss(X, Z)

print("R is ", R)

Result = X.dot(R)
print("X * R = ", Result)

In [8]:
def get_minor(matrix, row, col):
    # Получаем минор матрицы, исключая заданную строку и столбец
    return [row[:col] + row[col + 1:] for row in (matrix[:row] + matrix[row + 1:])]

def matrix_determinant(matrix):
    # Базовый случай: определитель 1x1 матрицы равен самому элементу
    if len(matrix) == 1:
        return matrix[0][0]
    
    # Базовый случай для 2x2 матрицы
    if len(matrix) == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    
    determinant = 0
    for col in range(len(matrix)):
        # Используем разложение по первой строке
        cofactor = (-1) ** col  # чередуем знаки
        minor = get_minor(matrix, 0, col)  # находим минор для текущего элемента
        determinant += cofactor * matrix[0][col] * matrix_determinant(minor)
    
    return determinant

def is_positive(A):
    for n in range(1, len(A) + 1):
        # Получаем подматрицу размером n x n
        sub_matrix = [row[:n] for row in A[:n]]
        # Вычисляем определитель главного минора
        main_minor_det = matrix_determinant(sub_matrix)
        if main_minor_det <= 0:
            return -1
    return 1

In [9]:
def LU(a, f):
    
    A = copy.deepcopy(a)
    F = copy.deepcopy(f)
    
    if (is_positive(A) == -1):
        return

    size_A = len(A)

    for i in range(1, size_A):
        for k in range(0, i):
            A[i][k] = A[i][k]/A[k][k]
            for j in range(k+1, size_A):
                A[i][j] = A[i][j] - A[i][k] * A[k][j]

    L = np.eye(size_A)
    U = np.zeros((size_A,size_A))

    for i in range(1, size_A):
        for j in range(i):
            L[i][j] = A[i][j]
            
    for i in range(0, size_A):
        for j in range(i, size_A):
            U[i][j] = A[i][j]

# Обратный ход для Ly = F -> ищем y

    Y = np.ones((size_A, 1))
            
    for i in range(0, size_A):
        Y[i] = F[i]
        for j in range(0, i):
            Y[i] -= Y[j] * L[i][j]

# Обратный ход для Ux = Y -> ищем x
    
    X  = np.ones((size_A, 1))

    for i in range(size_A - 1, -1, -1):
        X[i] = Y[i]
        for j in range(i + 1, size_A):
            X[i] -= X[j] * U[i][j]
        X[i] /= U[i][i]
        
    return X

In [ ]:
A = np.random.rand(4,4) * 100

f = np.zeros((4, 1))
for i in range(4): f[i] = i + 1

A = np.round(A)

X = LU(A, f)


print("X is ", X)

Result = A.dot(X)
print("A * X = ", Result)

In [15]:
X = np.random.rand(500,500)

Z = np.zeros((500, 1))
for i in range(500): Z[i] = i + 1


print("X is ", X)
print("Z is ", Z)

R = LU(X, Z)

if(R == None):
    print(None)
else:
    print("R is ", R)

    Result = X.dot(R)
    print("X * R = ", Result)   





X is  [[0.544566 0.678260 0.584654 ... 0.168851 0.016857 0.750311]
 [0.036173 0.618207 0.996082 ... 0.992306 0.895071 0.331799]
 [0.732726 0.955166 0.491762 ... 0.046445 0.104839 0.309806]
 ...
 [0.308627 0.357879 0.100262 ... 0.780375 0.448017 0.427003]
 [0.729517 0.381468 0.431789 ... 0.175465 0.327137 0.176863]
 [0.006649 0.845619 0.045134 ... 0.316757 0.432785 0.315650]]
Z is  [[1.000000]
 [2.000000]
 [3.000000]
 [4.000000]
 [5.000000]
 [6.000000]
 [7.000000]
 [8.000000]
 [9.000000]
 [10.000000]
 [11.000000]
 [12.000000]
 [13.000000]
 [14.000000]
 [15.000000]
 [16.000000]
 [17.000000]
 [18.000000]
 [19.000000]
 [20.000000]
 [21.000000]
 [22.000000]
 [23.000000]
 [24.000000]
 [25.000000]
 [26.000000]
 [27.000000]
 [28.000000]
 [29.000000]
 [30.000000]
 [31.000000]
 [32.000000]
 [33.000000]
 [34.000000]
 [35.000000]
 [36.000000]
 [37.000000]
 [38.000000]
 [39.000000]
 [40.000000]
 [41.000000]
 [42.000000]
 [43.000000]
 [44.000000]
 [45.000000]
 [46.000000]
 [47.000000]
 [48.000000]
 

ValueError: operands could not be broadcast together with shapes (0,) (2,) 

In [10]:
def yakobi(a, f, x_0):
    
    A = copy.deepcopy(a)
    b = copy.deepcopy(f)
    X_0 = copy.deepcopy(x_0)
    
    size = len(A)
    
    D = np.zeros((size,size))  
    E = np.zeros((size,size))
    F = np.zeros((size,size))
    
    for i in range(size):
        D[i][i] = A[i][i]
        
    for i in range(size):
        for j in range(i):
            E[i][j] = -1 * A[i][j]
            
    for i in range(size):
        for j in range(i+1, size):
            F[i][j] = -1 * A[i][j]
            
    print(D)
    print(E)
    print(F)
    print(D - E - F)
    
    
    X_prev = copy.deepcopy(x_0)
    X_new = copy.deepcopy(x_0)
    
    k = 0
    while( k < 100):
        for i in range(size):
            X_new[i] = 0
            for j in range(size):
                X_new[i] = X_new[i] + A[i][j] * X_prev[j]
                
            X_new[i] = (b[i] - X_new[i]) / A[i][i]
            
        for index in range(size):
            X_prev[index] = X_new[index]
            
        k += 1
        
    return X_new

In [ ]:
A = np.array([[1,2,3],[4,5,6],[7,8,9]])

U = yakobi(A, [1,2,3], [[0],[0],[0]])

print("U is \n", U)


